# Задание к теме «Гибридные рекомендательные системы»

Задание

Вопросы по заданию

Преподаватель: Наталья Баданина, Юлия Пономарева, Егор Шишковец

Что нужно делать?

    * Датасет ml-latest.
    * Вспомнить подходы, которые мы разбирали.
    * Выбрать понравившийся подход к гибридным системам.
    * Написать свою.

In [171]:
import pandas as pd

In [172]:
links = pd.read_csv('ml-latest-small/links.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')
tags = pd.read_csv('ml-latest-small/tags.csv')

In [173]:
print('links')
print(links.head(1))

print('movies')
print(movies.head(1))

print('ratings')
print(ratings.head(1))

print('tags')
print(tags.head(1))

links
   movieId  imdbId  tmdbId
0        1  114709   862.0
movies
   movieId             title                                       genres
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy
ratings
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
tags
   userId  movieId    tag   timestamp
0       2    60756  funny  1445714994


In [174]:
df = movies.merge(ratings, how="left", on="movieId")
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [175]:
df["title"].nunique()

9737

In [176]:
df["title"].value_counts().head()

Forrest Gump (1994)                 329
Shawshank Redemption, The (1994)    317
Pulp Fiction (1994)                 307
Silence of the Lambs, The (1991)    279
Matrix, The (1999)                  278
Name: title, dtype: int64

In [177]:
# Перейдем к фильмам с менее чем 10 рецензиями:
comment_counts = pd.DataFrame(df["title"].value_counts())
rare_movies = comment_counts[comment_counts["title"] <= 10].index

# получим доступ к фильмам с более чем 10 обзоров:
common_movies = df[~df["title"].isin(rare_movies)]
common_movies.shape

(79640, 6)

In [178]:
common_movies["title"].nunique()

2121

In [179]:
# создадим пользовательский Df:
user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")

user_movie_df.shape

(610, 2121)

In [180]:
user_movie_df.head(10)

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),...,Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [181]:
# item-based:
movie_name = "Matrix, The (1999)"
movie_name = user_movie_df[movie_name]
user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)

title
Matrix, The (1999)                                  1.000000
Nixon (1995)                                        1.000000
Murder in the First (1995)                          1.000000
Circle of Friends (1995)                            0.983739
American in Paris, An (1951)                        0.968330
Wings of Desire (Himmel über Berlin, Der) (1987)    0.902914
Pacific Rim (2013)                                  0.887852
Wyatt Earp (1994)                                   0.859338
Michael Collins (1996)                              0.814381
Persuasion (1995)                                   0.813326
dtype: float64

In [182]:
random_user = 2

In [183]:
# создадим dataset для пользователя 2:
random_user_df = user_movie_df[user_movie_df.index == random_user]
random_user_df

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),...,Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN


In [184]:
movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()
movies_watched

['Collateral (2004)',
 'Dark Knight Rises, The (2012)',
 'Dark Knight, The (2008)',
 'Departed, The (2006)',
 'Django Unchained (2012)',
 'Ex Machina (2015)',
 'Exit Through the Gift Shop (2010)',
 'Girl with the Dragon Tattoo, The (2011)',
 'Gladiator (2000)',
 'Good Will Hunting (1997)',
 'Inception (2010)',
 'Inglourious Basterds (2009)',
 'Inside Job (2010)',
 'Interstellar (2014)',
 'Kill Bill: Vol. 1 (2003)',
 'Mad Max: Fury Road (2015)',
 'Shawshank Redemption, The (1994)',
 'Shutter Island (2010)',
 'Step Brothers (2008)',
 'Talladega Nights: The Ballad of Ricky Bobby (2006)',
 'Tommy Boy (1995)',
 'Town, The (2010)',
 'Warrior (2011)',
 'Whiplash (2014)',
 'Wolf of Wall Street, The (2013)',
 'Zombieland (2009)']

In [185]:
user_movie_df.loc[user_movie_df.index == random_user, user_movie_df.columns == "Young Guns (1988)"]

title,Young Guns (1988)
userId,
2.0,NaN


In [186]:
# Сколько фильмов посмотрел пользователь:
len(movies_watched)

26

In [187]:
movies_watched_df = user_movie_df[movies_watched]
movies_watched_df.head()
movies_watched_df.shape

(610, 26)

In [188]:
# информация о том, сколько фильмов в общей сложности посмотрел каждый пользователь:
user_movie_count = movies_watched_df.T.notnull().sum()

user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["userId","movie_count"]
user_movie_count.head()

,userId,movie_count
0,1.0,2
1,2.0,26
2,3.0,0
3,4.0,1
4,5.0,1


In [189]:
user_movie_count[user_movie_count["movie_count"] == 2].count()

userId         73
movie_count    73
dtype: int64

In [190]:
perc = len(movies_watched) * 60 / 100
perc

15.6

In [191]:
# Люди, посмотревшие более 60% фильмов:
users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]
users_same_movies.count()

19

In [192]:
# Объединим данные пользователя 2 и похожих пользователей:
final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
                      random_user_df[movies_watched]])

final_df.shape
final_df.T.corr()

userId,2.0,18.0,62.0,68.0,105.0,111.0,249.0,274.0,279.0,298.0,305.0,352.0,414.0,448.0,495.0,560.0,561.0,599.0,610.0,2.0
userId,,,,,,,,,,,,,,,,,,,,
2.0,1.000000,-4.764558e-01,-0.162224,8.005616e-02,-0.270718,0.370150,0.145969,0.044739,-0.350438,0.269900,-0.423321,-0.287168,-0.300467,6.447258e-02,-7.265477e-02,0.378681,0.136169,-0.016021,0.623288,1.000000
18.0,-0.476456,1.000000e+00,0.162360,2.138963e-01,0.573045,-0.198757,0.156174,0.189389,0.340760,0.139716,0.059817,0.101073,0.710910,-1.958674e-16,3.061384e-02,-0.211560,0.177879,0.103139,-0.383514,-0.476456
62.0,-0.162224,1.623599e-01,1.000000,1.301522e-01,-0.167360,-0.310859,-0.056775,0.430875,-0.027039,0.293389,0.139927,-0.339400,0.302256,-4.305069e-02,6.244998e-01,0.089803,-0.196702,0.390498,0.093377,-0.162224
68.0,0.080056,2.138963e-01,0.130152,1.000000e+00,-0.241584,-0.299164,0.306448,0.438529,0.198108,0.253065,-0.119572,-0.391935,0.478850,2.694351e-01,3.205215e-16,0.649396,0.380636,0.131787,0.077789,0.080056
105.0,-0.270718,5.730448e-01,-0.167360,-2.415838e-01,1.000000,0.264927,0.203737,-0.055604,0.068657,-0.065029,-0.022476,0.526607,0.384148,2.524598e-01,-8.863600e-02,-0.146254,-0.149786,-0.142689,-0.214217,-0.270718
111.0,0.370150,-1.987572e-01,-0.310859,-2.991642e-01,0.264927,1.000000,0.185782,-0.312253,0.125553,-0.165768,0.079341,0.349790,-0.508304,-3.636832e-01,-3.583360e-01,0.088184,0.192643,-0.465937,0.072938,0.370150
249.0,0.145969,1.561738e-01,-0.056775,3.064482e-01,0.203737,0.185782,1.000000,0.750136,-0.041204,-0.113095,-0.307692,0.326183,0.134292,2.297506e-01,-4.568322e-01,0.107749,-0.094915,-0.038068,0.039546,0.145969
274.0,0.044739,1.893885e-01,0.430875,4.385290e-01,-0.055604,-0.312253,0.750136,1.000000,-0.054554,0.412043,0.253046,0.472456,0.442184,5.060072e-01,4.719399e-01,0.373306,0.047873,0.546897,-0.051243,0.044739
279.0,-0.350438,3.407597e-01,-0.027039,1.981080e-01,0.068657,0.125553,-0.041204,-0.054554,1.000000,-0.637895,-0.282340,0.094749,0.286251,-7.022866e-01,-3.278050e-01,-0.451229,-0.247446,-0.287274,-0.339705,-0.350438


In [193]:
#корреляция для всех пользователей:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()
corr_df.head()

,user_id_1,user_id_2,corr
0,448.0,279.0,-0.702287
1,279.0,298.0,-0.637895
2,414.0,111.0,-0.508304
3,2.0,18.0,-0.476456
4,2.0,18.0,-0.476456


In [194]:
# Пользователи с корреляцией %65 или более с 2 пользователем:
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][
    ["user_id_2", "corr"]].reset_index(drop=True)

top_users = top_users.sort_values(by='corr', ascending=False)
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)
top_users.head()

,userId,corr
0,2.0,1.0


In [195]:
# посмотрим рейтинги пользователей:
top_users_ratings = top_users.merge(ratings[["userId", "movieId", "rating"]], how='inner')

top_users_ratings = top_users_ratings[top_users_ratings["userId"] != random_user]
top_users_ratings.head()

,userId,corr,movieId,rating


In [196]:
# Найдем 5 рекомендаций.

top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})

recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})
recommendation_df = recommendation_df.reset_index()
recommendation_df.head()

,movieId,weighted_rating


In [197]:
# взвешенный рейтинг больше 4:
recommendation_df[recommendation_df["weighted_rating"] > 3.7]

movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.7].sort_values("weighted_rating", ascending=False)

movies_to_be_recommend.merge(movies[["movieId", "title"]])

#Топ 5 лучших фильмов:
movies_to_be_recommend.merge(movies[["movieId", "title"]])[:5]

,weighted_rating,movieId,title


In [198]:
# ▪ 5 предложений user-based алгоритма
# ▪ 5 предложений item-based алгоритма

# Последний фильм с высоким рейтингом пользователя 2:

user = 2
movie_id = ratings[(ratings["userId"] == user) & (ratings["rating"] == 5.0)].sort_values(by="timestamp", ascending=False)["movieId"][0:1].values[0]
movie_id

122882

In [199]:
# ▪ 5 предложений user-based алгоритма
movies_to_be_recommend.merge(movies[["movieId", "title"]])[:5]['title'].to_list()

[]

In [200]:
# ▪ 5 предложений item-based алгоритма
movie_name = movies[movies['movieId'] == movie_id]['title'].values[0]
movie_name = user_movie_df[movie_name]
moveis_from_item_based = user_movie_df.corrwith(movie_name).sort_values(ascending=False)
moveis_from_item_based[1:6].index.to_list()

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


['Boiler Room (2000)',
 'Nine Months (1995)',
 "What's Love Got to Do with It? (1993)",
 'Dune (1984)',
 'Dumb and Dumberer: When Harry Met Lloyd (2003)']